### Introduction

Here you'll learn to build models using Catboost, Lightgbm and NaiveBayes algorithm in Python. Given the text classification problem, you'll also learn to clean data, create bag of words matrix, tf-idf matrix. 

On top of what's done here, next you can create a simple voting ensemble from the predictions generated from these models here.

In [1]:
# Load Libraries
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer

In [2]:
# load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [4]:
# function to clean data

stops = set(stopwords.words("english"))
def cleanData(text, lowercase = False, remove_stops = False, stemming = False):
    txt = str(text)
    txt = re.sub(r'[^A-Za-z0-9\s]',r'',txt)
    txt = re.sub(r'\n',r' ',txt)
    
    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()])
        
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    
    if stemming:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in txt.split()])

    return txt

In [5]:
## join data
test['Is_Response'] = np.nan
alldata = pd.concat([train, test]).reset_index(drop=True)

In [6]:
# clean description
alldata['Description'] = alldata['Description'].map(lambda x: cleanData(x, lowercase=True, remove_stops=True, stemming=True))

In [7]:
# initialise the functions - we'll create separate models for each type.
countvec = CountVectorizer(analyzer='word', ngram_range = (1,1), min_df=150, max_features=500)
tfidfvec = TfidfVectorizer(analyzer='word', ngram_range = (1,1), min_df = 150, max_features=500)

In [8]:
# create features
bagofwords = countvec.fit_transform(alldata['Description'])
tfidfdata = tfidfvec.fit_transform(alldata['Description'])

In [9]:
# label encode categorical features in data given
cols = ['Browser_Used','Device_Used']

for x in cols:
    lbl = LabelEncoder()
    alldata[x] = lbl.fit_transform(alldata[x])

In [10]:
# create dataframe for features
bow_df = pd.DataFrame(bagofwords.todense())
tfidf_df = pd.DataFrame(tfidfdata.todense())

In [11]:
# set column names
bow_df.columns = ['col'+ str(x) for x in bow_df.columns]
tfidf_df.columns = ['col' + str(x) for x in tfidf_df.columns]

In [12]:
# create separate data frame for bag of words and tf-idf

bow_df_train = bow_df[:len(train)]
bow_df_test = bow_df[len(train):]

tfid_df_train = tfidf_df[:len(train)]
tfid_df_test = tfidf_df[len(train):]

In [13]:
# split the merged data file into train and test respectively
train_feats = alldata[~pd.isnull(alldata.Is_Response)]
test_feats = alldata[pd.isnull(alldata.Is_Response)]

In [14]:
### set target variable

train_feats['Is_Response'] = [1 if x == 'happy' else 0 for x in train_feats['Is_Response']]

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [15]:
# merge count (bag of word) features into train
train_feats1 = pd.concat([train_feats[cols], bow_df_train], axis = 1)
test_feats1 = pd.concat([test_feats[cols], bow_df_test], axis=1)

test_feats1.reset_index(drop=True, inplace=True)

In [16]:
# merge into a new data frame with tf-idf features
train_feats2 = pd.concat([train_feats[cols], tfid_df_train], axis=1)
test_feats2 = pd.concat([test_feats[cols], tfid_df_test], axis=1)

### NaiveBayes

In [17]:
# let's check cross validation score of the model
# cv score acts a unbiased estimate of models accuracy on unseen data

mod1 = GaussianNB()
target = train_feats['Is_Response']

In [18]:
## Naive Bayes 1
print(cross_val_score(mod1, train_feats1, target, cv=5, scoring=make_scorer(accuracy_score)))

[ 0.77208526  0.76110968  0.76753147  0.76663242  0.77626509]


In [19]:
## Naive Bayes 2 - tfidf is giving higher CV score
print(cross_val_score(mod1, train_feats2, target, cv=5, scoring=make_scorer(accuracy_score)))

[ 0.80906523  0.81518109  0.80901618  0.81312612  0.80349345]


In [20]:
# make our first set of predictions

clf1 = GaussianNB()
clf1.fit(train_feats1, target)

clf2 = GaussianNB()
clf2.fit(train_feats2, target)

GaussianNB(priors=None)

In [21]:
preds1 = clf1.predict(test_feats1)
preds2 = clf2.predict(test_feats2)

In [22]:
def to_labels(x):
    if x == 1:
        return "happy"
    return "not_happy"

In [23]:
sub1 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds1})
sub1['Is_Response'] = sub1['Is_Response'].map(lambda x: to_labels(x))

In [24]:
sub2 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds2})
sub2['Is_Response'] = sub2['Is_Response'].map(lambda x: to_labels(x))

In [25]:
sub1 = sub1[['User_ID', 'Is_Response']]
sub2 = sub2[['User_ID', 'Is_Response']]

In [26]:
## write submission files
sub1.to_csv('submissions/sub1_cv.csv', index=False)
sub2.to_csv('submissions/sub2_tf.csv', index=False)

### LightGBM - 1

We are prefering lightgbm over xgboost because of its speed. <br />
In this model, we'll use count features for model training.

In [27]:
import lightgbm as lgb

In [28]:
# set the data in format lgb accepts
d_train = lgb.Dataset(train_feats1, label = target)

In [29]:
## set parameters
## you can tune the parameters can try to better score

params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05, 
    'max_depth': 7, 
    'num_leaves': 21, 
    'feature_fraction': 0.3, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 5}

In [30]:
lgb_cv = lgb.cv(params, d_train, num_boost_round=500, nfold= 5, shuffle=True, stratified=True, verbose_eval=20, early_stopping_rounds=40)

[20]	cv_agg's binary_error: 0.20148 + 0.00422251
[40]	cv_agg's binary_error: 0.182857 + 0.00339422
[60]	cv_agg's binary_error: 0.169757 + 0.00235841
[80]	cv_agg's binary_error: 0.160485 + 0.00264504
[100]	cv_agg's binary_error: 0.154269 + 0.00345369
[120]	cv_agg's binary_error: 0.1498 + 0.00343291
[140]	cv_agg's binary_error: 0.145356 + 0.00220508
[160]	cv_agg's binary_error: 0.142145 + 0.00335011
[180]	cv_agg's binary_error: 0.140116 + 0.00250729
[200]	cv_agg's binary_error: 0.137804 + 0.00276994
[220]	cv_agg's binary_error: 0.136571 + 0.00330318
[240]	cv_agg's binary_error: 0.133926 + 0.00311789
[260]	cv_agg's binary_error: 0.133078 + 0.00292937
[280]	cv_agg's binary_error: 0.131742 + 0.00257978
[300]	cv_agg's binary_error: 0.13015 + 0.00240762
[320]	cv_agg's binary_error: 0.129662 + 0.00236336
[340]	cv_agg's binary_error: 0.129148 + 0.00282004
[360]	cv_agg's binary_error: 0.128506 + 0.0025436
[380]	cv_agg's binary_error: 0.127376 + 0.00250987
[400]	cv_agg's binary_error: 0.126888 + 

In [31]:
## get nround value which hd lowest error
nround = lgb_cv['binary_error-mean'].index(np.min(lgb_cv['binary_error-mean']))

In [32]:
## train the model
model = lgb.train(params, d_train, num_boost_round=nround)

In [33]:
## make predictions
preds = model.predict(test_feats1)

In [34]:
# make submission

def to_labels(x):
    if x > 0.66:  # cutoff - you can change it and see if accuracy improves or plot AUC curve. 
        return "happy"
    return "not_happy"

sub3 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds})
sub3['Is_Response'] = sub3['Is_Response'].map(lambda x: to_labels(x))
sub3 = sub3[['User_ID','Is_Response']]
sub3.to_csv('submissions/sub3_lgb.csv', index=False) # 0.85518

### LightGBM - 2

In this model, we'll use tf-idf features for model training.

In [35]:
# set data format
d_train = lgb.Dataset(train_feats2, label = target)

In [36]:
# same parameters as above
params = {'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_error',
    'learning_rate': 0.05, 
    'max_depth': 5, 
    'num_leaves': 11,
    'feature_fraction': 0.3, 
    'bagging_fraction': 0.8, 
    'bagging_freq': 5}

In [37]:
## do cross validation to find nround i.e. at this round (iteration) we can expect lowest error
lgb_cv = lgb.cv(params, d_train, num_boost_round=500, nfold= 5, shuffle=True, stratified=True, verbose_eval=20, early_stopping_rounds=40)

[20]	cv_agg's binary_error: 0.211266 + 0.0046732
[40]	cv_agg's binary_error: 0.194442 + 0.00350919
[60]	cv_agg's binary_error: 0.182883 + 0.00241633
[80]	cv_agg's binary_error: 0.173148 + 0.00260586
[100]	cv_agg's binary_error: 0.165442 + 0.0029773
[120]	cv_agg's binary_error: 0.158764 + 0.00285086
[140]	cv_agg's binary_error: 0.153678 + 0.00259374
[160]	cv_agg's binary_error: 0.149697 + 0.00240968
[180]	cv_agg's binary_error: 0.147282 + 0.00253651
[200]	cv_agg's binary_error: 0.144868 + 0.0030508
[220]	cv_agg's binary_error: 0.142299 + 0.0027785
[240]	cv_agg's binary_error: 0.140861 + 0.00353668
[260]	cv_agg's binary_error: 0.138703 + 0.00303762
[280]	cv_agg's binary_error: 0.137393 + 0.00319081
[300]	cv_agg's binary_error: 0.136058 + 0.00307116
[320]	cv_agg's binary_error: 0.135416 + 0.00265982
[340]	cv_agg's binary_error: 0.134234 + 0.00275393
[360]	cv_agg's binary_error: 0.13295 + 0.00286717
[380]	cv_agg's binary_error: 0.132693 + 0.00310614
[400]	cv_agg's binary_error: 0.131871 + 

In [38]:
# get nround value
nround = lgb_cv['binary_error-mean'].index(np.min(lgb_cv['binary_error-mean']))

In [39]:
# train model
model = lgb.train(params, d_train, num_boost_round=nround)

In [40]:
# make prediction
preds = model.predict(test_feats2)

In [41]:
# make submission

def to_labels(x):
    if x > 0.66:
        return "happy"
    return "not_happy"

sub4 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds})
sub4['Is_Response'] = sub4['Is_Response'].map(lambda x: to_labels(x))
sub4 = sub4[['User_ID','Is_Response']]
sub4.to_csv('submissions/sub4_lgb.csv', index=False) # 0.84925

### CatBoost

Catboost is a new package recently launched by Yandex. It is said that it works well when the data has many categorical features. We'll use it on count data and see it our model improves.

In [42]:
## import library
from catboost import CatBoostClassifier,cv, Pool

In [43]:
## catboost accepts categorical columns as a list of column numbers. In this data, all columns are categorical
cat_cols = [x for x in range(502)] ## 502 == train_feats1.shape[1] 

In [44]:
## set parameters
## you can refer the parameters here: https://tech.yandex.com/catboost/doc/dg/concepts/python-reference_parameters-list-docpage/#python-reference_parameters-list
param = {
    'use_best_model':True,
    'loss_function':'CrossEntropy',
    'eval_metric':'Accuracy',
    'iterations':1000,
    'depth':6,
    'learning_rate':0.03,
    'rsm':0.3,
    'random_seed':2017,
}

In [45]:
## for doing cross validation, set data in Pool format
my_dt =  Pool(train_feats1, 
           label=target,
           cat_features=cat_cols,
           column_description=None,
           delimiter='\t',
           has_header=None,
           weight=None, 
           baseline=None,
           feature_names=None,
           thread_count=1)

In [46]:
## run cv to get best iteration
ctb_cv = cv(param, my_dt, fold_count=5, partition_random_seed=2017)

/usr/local/lib/python3.5/dist-packages/catboost/core.py:1425: UserWarning: Parameter "use_best_model" has no effect in cross-validation and is ignored
  warnings.warn('Parameter "use_best_model" has no effect in cross-validation and is ignored')


In [47]:
# fetch best round
best_round = ctb_cv['b\'Accuracy\'_test_avg'].index(np.max(ctb_cv['b\'Accuracy\'_test_avg']))

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
## define the classifer model
model = CatBoostClassifier(iterations=best_round, learning_rate=0.03,rsm = 0.3 ,depth=6, eval_metric='Accuracy', random_seed=2017)

In [ ]:
## train model
model.fit(my_dt)

In [ ]:
## make predictions
preds = model.predict(test_feats1)

In [ ]:
## make submission
sub5 = pd.DataFrame({'User_ID':test.User_ID, 'Is_Response':preds})
sub5['Is_Response'] = ['happy' if x == 1 else 'not_happy' for x in sub5['Is_Response']]
sub5 = sub5[['User_ID','Is_Response']]
sub5.to_csv('submissions/sub5_cb.csv', index=False)